In [4]:
import re
import os
from collections import defaultdict

from PyPDF2 import PdfWriter, PdfReader
# import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import pandas as pd

In [5]:
df = pd.read_csv("paragraphs.csv.zip")
df.head()

/tmp/ipykernel_3347/4051087380.py:1: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragraphs.csv.zip")


,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0.0,1 ММК,ммк
1,1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...
2,2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,2.0,www.mmk.ru,NaN
3,3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3.0,СОДЕРЖАНИЕ,содержание
4,4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...


In [6]:
df[df["company"]=="BIOCAD"]

,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text
153376,153376,BIOCAD,"2021, 2020",Здравоохранение и спорт,ОУР,0.0,2021—2022,NaN
153377,153377,BIOCAD,"2021, 2020",Здравоохранение и спорт,ОУР,1.0,Главное — качество жизни людей Отчет об устойч...,главное качество жизнь человек отчёт устойчивы...
153378,153378,BIOCAD,"2021, 2020",Здравоохранение и спорт,ОУР,2.0,Оглавление Об отчете,оглавление о отчёт
153379,153379,BIOCAD,"2021, 2020",Здравоохранение и спорт,ОУР,3.0,4,NaN
153380,153380,BIOCAD,"2021, 2020",Здравоохранение и спорт,ОУР,4.0,1,NaN
...,...,...,...,...,...,...,...,...
975324,975324,BIOCAD,"2019, 2018",Здравоохранение и спорт,ОУР,1492.0,Соответствие отчета Руководству GRI Standards,соответствие отчёт руководство
975325,975325,BIOCAD,"2019, 2018",Здравоохранение и спорт,ОУР,1493.0,Контакты и обратная связь Мы хотим с каждым г...,контакт обратный связь мы хотеть каждый год ул...
975326,975326,BIOCAD,"2019, 2018",Здравоохранение и спорт,ОУР,1494.0,103,NaN
975327,975327,BIOCAD,"2019, 2018",Здравоохранение и спорт,ОУР,1495.0,Отчет об устойчивом развитии BIOCAD 2019/2020,отчёт устойчивый развитие


In [4]:
REPORTS_DIRECTORY = "reports"
PAGES_DIRECTORY = "pages"
PAGES_TXT_DIRECTORY = "pages_txt"

RE_DERIVED = re.compile("\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile("[а-яА-Я]+")
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+")

stopwords_ru = stopwords.words("russian")

In [5]:
def reports_to_pages():
    errors = []
    for report in tqdm(os.listdir(REPORTS_DIRECTORY)):
        if not report.endswith(".pdf"):
            continue
        try:
            input_pdf = PdfReader(open(f"{REPORTS_DIRECTORY}/{report}", "rb"))
        except Exception:
            errors.append(report)
            continue
        _, report = report.rsplit("_", maxsplit=1)
        report = report.rstrip(".pdf")
        for index, page in enumerate(input_pdf.pages):
            output = PdfWriter()
            output.add_page(page)
            with open(f"{PAGES_DIRECTORY}/{report}_{index}.pdf", "wb") as outputStream:
                output.write(outputStream)
    print(*errors, sep=" ")

In [7]:
reports_to_pages()

  4%|███████▎                                                                                                                                                                        | 4/96 [00:46<17:55, 11.70s/it]


KeyboardInterrupt: 

In [8]:
for report_page_name in tqdm(os.listdir(PAGES_DIRECTORY)):
    os.rename(f"{PAGES_DIRECTORY}/{report_page_name}", f'{PAGES_DIRECTORY}/{report_page_name.replace(".pdf_", "_")}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 477/477 [00:00<00:00, 262694.72it/s]


In [9]:
os.listdir(PAGES_DIRECTORY)[:5]

['0_10.pdf', '74_19.pdf', '0_46.pdf', '65_103.pdf', '74_69.pdf']

In [10]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [11]:
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}

In [24]:
PAGES_DIRECTORY_KEK = os.listdir(PAGES_DIRECTORY).copy()

In [27]:
def parse_pages_to_texts():
    errors = []
    for report_page_name in tqdm(PAGES_DIRECTORY_KEK):
        if not report_page_name.endswith(".pdf"):
            continue
        try:
            report_page = textract.process(f"{PAGES_DIRECTORY}/{report_page_name}")
        except Exception:
            errors.append(report_page_name)
            continue
        print(report_page.decode())
        report_page = report_page.decode().replace("\n", " ")
        report_page = RE_DERIVED.sub("", report_page)
        report_page_lst = word_tokenize(report_page)
        # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
        tokens = []
        for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
            if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                word_ = word_.strip()
                word_ = morph.normal_forms(word_)[0]
                tokens.append(word_)
        russian_report_page_txt = " ".join(tokens)
        if not russian_report_page_txt:
            continue
        with open(f'{PAGES_TXT_DIRECTORY}/{report_page_name.replace(".pdf", ".txt")}', "w") as f:
            f.write(russian_report_page_txt)
    print(*errors, sep="\n")

In [28]:
parse_pages_to_texts()

  0%|▋                                                                                                                                                                              | 2/477 [00:00<00:28, 16.79it/s]

География операционной деятельности

Россия
1. Ямало-Ненецкий автономный округ
2. Ханты-Мансийский автономный округ
3. Ненецкий автономный округ
4. Сахалинская область
5. Республика Саха
6. Красноярский край
7. Томская область
8. Кемеровская область
9. Иркутская область
10. Тюменская область
11. Вологодская область

12. Волгоградская область
13. Мурманская область
14. Забайкальский край
15. Магаданская область
16. Республика Бурятия
17. Чукотский автономный округ
18. Камчатский край
19. Краснодарский край
20. Ленинградская область
21. Московская область

Казахстан
Астана
Вьетнам
Ханой
Вунгтау
Кыргызстан
Бишкек
Узбекистан
Ташкент

Учебный центр
Швейный цех

11



ОТЧЕТ О КОРПОРАТИВНОЙ ОТВЕТСТВЕННОСТИ
ЗА 2015 И 2016 ФИНАНСОВЫЕ ГОДЫ

Наши люди

20

НАШИ
ЛЮДИ
Расшифровка генома человека
В 2003 году была завершена расшифровка человеческого
генома, хотя анализ отдельных его участков еще продолжается. Геном любого отдельно взятого организма уникален, поэтому определение последовательности че

  1%|██▏                                                                                                                                                                            | 6/477 [00:00<00:40, 11.50it/s]

ОТЧЕТ О КОРПОРАТИВНОЙ ОТВЕТСТВЕННОСТИ
ЗА 2015 И 2016 ФИНАНСОВЫЕ ГОДЫ

Информация об отчете

70

Обзор данных

ТЕКУЧЕСТЬ КАДРОВ
Сотрудники, покинувшие компанию в 2016 финансовом году, с распределением
по должности и возрасту
Должность

До 30 лет

31-50 лет

Сотрудники, покинувшие компанию в 2015 финансовом году, с распределением
по должности и возрасту

Старше 51 года

Должность

Количество

Доля

Количество

Доля

Количество

Доля

Партнеры

0

0%

18

1,84%

4

0,41%

Партнеры

Сотрудники сервисных линий от уровня специалистов
до директоров

426

43,47%

92

9,39%

4

0,41%

Сотрудники сервисных линий от уровня специалистов
до директоров

Сотрудники отделов поддержки бизнеса от уровня
специалистов до
директоров

21

2,14%

22

2,24%

1

0,10%

Административный
персонал

9

0,92%

3

0,31%

0

Выпускники университетов
и/или магистры

181

18,47%

1

0,10%

Прочие сотрудники

156

15,92%

31

ИТОГО

793

80,92%

167

Старше 51 года

Количество

Доля

Количество

Доля

0%

13

1,11%

2




  2%|██▉                                                                                                                                                                            | 8/477 [00:00<00:52,  8.99it/s]

...

  5%|████████▊                                                                                                                                                                     | 24/477 [00:03<01:23,  5.41it/s]

Концепция управления персоналом
Права человека

Эффективная мотивация и справедливое вознаграждение

ГК «ЦКМ» в своей деятельности придерживается Всеобщей декларации прав человека, Глобального
договора ООН, Конвенции МОТ, Трудового кодекса страны присутствия.

В ГК «ЦКМ» действует справедливая, прозрачная система вознаграждения и мотивации. Она связывает
индивидуальные цели сотрудников с целями всей компании.

Нормативно-правовые требования
Политика компании в области управления персоналом полностью соответствует трудовому
законодательству страны присутствия и обеспечивает определяемые законом права, льготы и выплаты
для всех сотрудников группы компаний.

Производственная безопасность и условия труда
«ЦКМ» стремится создать для своих сотрудников комфортную рабочую среду путем реализации
комплексной программы по охране труда и технике безопасности, постоянного мониторинга качества
условий труда и их улучшения, а также путем совершенствования рабочего процесса.

ГК «ЦКМ» следит за рынком

  5%|█████████                                                                                                                                                                     | 25/477 [00:03<01:11,  6.36it/s]

ПРИЛОЖЕНИЯ
ЭН+ ГРУП МКПАО
Примечания к консолидированной финансовой отчетности за 2019 год

будет признаваться по наилучшей оценке суммы, ожидаемой к оплате. Если руководство
считает, что существует вероятность того, что судебные разбирательства не приведут к
выплате компенсации третьим сторонам или если, в редких случаях, руководство не считает
возможным предоставить достаточно надежную оценку суммы, ожидаемой к оплате, то
резерв по какому-либо потенциальному обязательству в рамка разбирательства не создается,
однако рассматриваемые обстоятельства и неопределенности должны отражаться в составе
условных обязательств.
Оценка возможных результатов судебных разбирательств, а также сумма потенциального
обязательства содержит значимое суждение. Так как законы и нормативно-правовые акты во
многих странах, где функционирует Группа, продолжают изменяться, частично в отношении
налогообложения, прав на недропользование и защиты окружающей среды, уровень
неопределенностей, связанных с разбиратель

KeyboardInterrupt: 

In [21]:
a = len(os.listdir(PAGES_TXT_DIRECTORY))
b = len(os.listdir(PAGES_DIRECTORY))

In [18]:
1 - a / b

1.0

In [13]:
a, b

(118411, 129886)

In [14]:
a / b

0.9116532959672328

In [17]:
rep_words_freq = defaultdict(int)  # в скольких отчетах употребляется слово

In [18]:
for report in tqdm(os.listdir(PAGES_TXT_DIRECTORY)):
    if not report.endswith(".txt"):
        continue

    report_filename = f"{PAGES_TXT_DIRECTORY}/{report}"
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)

    report_words = set()
    for word in report_text_lst:
        if not word in report_words:
            report_words.add(word)
            rep_words_freq[word] += 1

100%|██████████████████████████████████| 118411/118411 [02:30<00:00, 785.50it/s]


In [19]:
amount_of_pages = len(os.listdir(PAGES_TXT_DIRECTORY))
rep_words_freq_percent = {k: v / amount_of_pages for (k, v) in rep_words_freq.items()}

In [20]:
rep_words_freq_percent

{'период': 0.2523414209828479,
 'летний': 0.01421320654331101,
 'межень': 0.0001182322588273049,
 'в': 0.6759929398451158,
 'обычный': 0.011029380716318586,
 'существенный': 0.13969141380446073,
 'влияние': 0.10435685873778619,
 'наблюдаться': 0.01087736781211205,
 'предел': 0.03423668409184957,
 'участок': 0.04562920674599488,
 'недропользование': 0.0022886387244428303,
 'добыча': 0.0625617552423339,
 'питьевой': 0.007085490368293486,
 'вода': 0.08693449088344833,
 'подземный': 0.022742819501566577,
 'скважина': 0.01932252915691954,
 'образоваться': 0.017135232368614403,
 'депрессионный': 1.6890322689614984e-05,
 'воронка': 0.00043070322858518214,
 'сведение': 0.07491702628978726,
 'общий': 0.3127749955662903,
 'количество': 0.19461874319108866,
 'забирать': 0.006958812948121374,
 'источник': 0.09598770384508196,
 'местонахождение': 0.006376096815329657,
 'таблица': 0.061218974588509514,
 'объём': 0.23328913698896217,
 'водопотребление': 0.012574845242418356,
 'тыс': 0.157569820371418

In [22]:
min(list(rep_words_freq_percent.values())), max(list(rep_words_freq_percent.values()))

(8.445161344807492e-06, 0.7362153853949379)

In [64]:
import numpy as np
import pandas as pd

In [86]:
v = np.array(list(rep_words_freq_percent.values()))
f"{np.percentile(v, 85):.6f}"

'0.000110'

In [78]:
between_ = set()
for key, value in tqdm(rep_words_freq_percent.items()):
    if value >= 0.000878 and value <= 0.3:
        between_.add(key)

100%|██████████████████████████████| 183749/183749 [00:00<00:00, 2749619.74it/s]


In [79]:
len(between_)

9177

In [65]:
pages_txt = {"report_num": [], "page_num": [], "text": []}

In [66]:
os.listdir(PAGES_TXT_DIRECTORY)[:5]

['341_187.txt', '1173_9.txt', '49_75.txt', '1253_99.txt', '942_64.txt']

In [82]:
for report in tqdm(os.listdir(PAGES_TXT_DIRECTORY)):
    if not report.endswith(".txt"):
        continue
    report_filename = f"{PAGES_TXT_DIRECTORY}/{report}"
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word in between_]
    if len(report_words) == 0:
        continue
    report_num, page_num = get_report_and_page_ids(report)
    pages_txt["report_num"].append(report_num)
    pages_txt["page_num"].append(page_num)
    pages_txt["text"].append(" ".join(report_words))

100%|██████████████████████████████████| 118411/118411 [02:27<00:00, 802.58it/s]


In [83]:
pages_df = pd.DataFrame.from_dict(pages_txt)
pages_df.head()

,report_num,page_num,text
0,341,187,период летний обычный период существенный влия...
1,1173,9,основной направление природоохранный экология ...
2,49,75,мнение заинтересованный сторона внутренний вне...
3,1253,99,действие сфера культура спор тот г. средство в...
4,942,64,л


In [84]:
pages_df.shape

(118157, 3)

In [85]:
pages_df.to_csv("report_pages_ds.csv")